In [1]:
import time
import logging
from kiteconnect import KiteTicker
logger = logging.getLogger()
logging.basicConfig(level=logging.DEBUG)
import datetime
from utils.RenkoUtil import *
from classes.DayEntry import Day
from classes.MinuteEntry import Minute
from classes.TradeEntry import Trade
from collections import deque
import thread

In [2]:
#initialize
money = 1
day = Day(0, money, False)
stepCount = 3
brickHeight = 0.05
renkoDeque = deque(maxlen=stepCount)
printRenkoDeque(renkoDeque)

DEBUG:root:size of queue: 0
DEBUG:root:empty queue


In [3]:
def tickScript(tick):
    verbose = False
    global day
    global renkoDeque
    global stepCount
    global brickHeight
    logging.info('Inside tickfunction that one')
    minute = Minute(tick['timestamp'], tick['last_price'])
    renkoDeque, newRenko = generateRenko(minute, renkoDeque, brickHeight, stepCount, verbose)
    printRenkoDeque(renkoDeque)
    while newRenko:
        if len(renkoDeque) == stepCount:
    #             if marketOpen <= minute.time < marketClose:
            logging.info("Boughtflag: {}".format(day.boughtFlag))
            if day.boughtFlag == 0:
                #market enter logic
                if getEmotion(renkoDeque) == stepCount:
                    #buy
                    day.boughtFlag = 1
                    day.currentTrade = Trade(minute.listPrice, minute.time, 0, 0, day.boughtFlag)
                    logging.info('Long trade {}'.format(day.currentTrade.enterPrice))

                elif getEmotion(renkoDeque) == -1 * stepCount:
                    #sell
                    day.boughtFlag = -1
                    day.currentTrade  = Trade(minute.listPrice, minute.time, 0, 0, day.boughtFlag)
                    logging.info('Short trade {}'.format(day.currentTrade.enterPrice))

            elif day.boughtFlag == 1:
                #market exit logic (when already bought)
                if renkoExitLogic(renkoDeque, stepCount, day.boughtFlag):
                    logging.info('exiting trade, enter: {} exit: {}'.format(day.currentTrade.enterPrice, minute.listPrice))
                    day.currentTrade.exitTrade('renkoExit', minute.time, minute.listPrice, verbose = True)
                    day.addTrade(day.currentTrade)
                    logging.info('Long exit')
            else:
                #market exit logic (when already sold)
                if renkoExitLogic(renkoDeque, stepCount, day.boughtFlag):
                    logging.info('exiting trade, enter: {} exit: {}'.format(day.currentTrade.enterPrice, minute.listPrice))
                    day.currentTrade.exitTrade('renkoExit', minute.time, minute.listPrice, verbose = True)
                    day.addTrade(day.currentTrade)
                    logging.info('Short exit')
        renkoDeque, newRenko = generateRenko(minute, renkoDeque, brickHeight, stepCount, verbose)
    #         elif minute.time >= marketClose and day.sellEndOfDay:
    #             if day.boughtFlag == 1:
    #change to currentTrade
    #                 trade.exitTrade('eod', minute.time, minute.closingPrice, verbose)
    #                 day.addTrade(trade)
    #             elif day.boughtFlag == -1:
    #                 trade.exitTrade('eod', minute.time, minute.closingPrice, verbose)
    #                 day.addTrade(trade)

In [4]:
# Initialise
kws = KiteTicker("5auxbio6wezhjhhb", "YTyOIbhkCTAbCNAxeOzdi2z77EDRxCJB")

def on_ticks(ws, ticks):
    # Callback to receive ticks.
#     print(time.time())
    logging.debug("Ticks: {}".format(ticks[0]['last_price']))
    try:
        thread.start_new_thread(tickScript, (ticks[0],))
    except:
        logging.debug('Unable to start that one thread')
def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
#     ws.subscribe([738561])
    ws.subscribe([6401])

    # Set RELIANCE to tick in `full` mode.
    ws.set_mode(ws.MODE_FULL, [6401])

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    #ws.stop()
    pass

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect(threaded=True)

DEBUG:root:Ticks: 120.0
INFO:root:Inside tickfunction that one
DEBUG:root:size of queue: 1


time: 2019-05-10 15:41:31, low: 120.0, high: 120.0, type: 0


DEBUG:root:Ticks: 120.0
INFO:root:Inside tickfunction that one
DEBUG:root:size of queue: 1


time: 2019-05-10 15:41:31, low: 120.0, high: 120.0, type: 0


DEBUG:root:Ticks: 120.0
INFO:root:Inside tickfunction that one
DEBUG:root:size of queue: 1


time: 2019-05-10 15:41:31, low: 120.0, high: 120.0, type: 0
